In [27]:
import xml.etree.ElementTree as ET

In [28]:
tree = ET.parse(r"Z:\Emma\hs\bb41dbf0-41ce-4913-ac11-9a37ce70c088\bb41dbf0-41ce-4913-ac11.xml")#ET.parse(r"W:\Images\Emma Westlund\hs\16b8fbc7-3b7c-4928-8fba-74219c7fdaca\16b8fbc7-3b7c-4928-8fba.xml")

In [29]:
root = tree.getroot()

In [30]:
#get namespace
namespace = root.tag.split('}')[0].strip('{') if '}' in root.tag else ''

ns = {'ns':namespace}

In [31]:
# Find channel order

channels = tree.findall('.//ns:Sequence/ns:Record/ns:Channel', ns)

channel_order = []
for channel in channels:
    channel_order.append(channel.text)

print(channel_order)

['Alexa 488', 'DAPI']


In [48]:
# Find position of images

# Step 1: Find Sublayouts- x and y coordinates of field for each well
# Step 2: Match well and sublayout ID
# Step 3: Save dict - key: well-id (ex, r01c01), value: [[x1,y1], [x2,y2] ...] for all fields

sublayouts = tree.findall('.//ns:Experiment/ns:Sublayouts/ns:Sublayout', ns)
wells = tree.findall('.//ns:Experiment/ns:MeasurementLayout/ns:Wells/ns:Well', ns)

well_layout = {}

for well in wells:
    col = well.find('ns:Col', ns).text
    row = well.find('ns:Row', ns).text

    well_name = "r"+row.zfill(2)+"c"+col.zfill(2)
    sublayout_id = int(well.find('ns:SublayoutID', ns).text)

    sublayout = sublayouts[sublayout_id-1]

    fields = sublayout.findall('ns:Field', ns)

    field_layout = []
    for field in fields:
        x = float(field.find('ns:X', ns).text)*(10**6)
        y = float(field.find('ns:Y', ns).text)*(10**6)
        field_layout.append([x,y])

    well_layout[well_name] = field_layout

print(well_layout)


{'r06c04': [[0.0, 0.0], [-1217.608107253909, 1217.6078744232655], [-608.8041700422764, 1217.6078744232655], [0.0, 1217.6078744232655], [608.8041700422764, 1217.6078744232655], [1217.6078744232655, 1217.6078744232655], [1217.6078744232655, 608.8041700422764], [608.8041700422764, 608.8041700422764], [0.0, 608.8041700422764], [-608.8041700422764, 608.8041700422764], [-1217.608107253909, 608.8041700422764], [-1217.608107253909, 0.0], [-608.8041700422764, 0.0], [608.8041700422764, 0.0], [1217.6078744232655, 0.0], [1217.6078744232655, -608.8041700422764], [608.8041700422764, -608.8041700422764], [0.0, -608.8041700422764], [-608.8041700422764, -608.8041700422764], [-1217.608107253909, -608.8041700422764], [-1217.608107253909, -1217.608107253909], [-608.8041700422764, -1217.608107253909], [0.0, -1217.608107253909], [608.8041700422764, -1217.608107253909], [1217.6078744232655, -1217.608107253909]], 'r02c02': [[0.0, 0.0], [-1217.608107253909, 1217.6078744232655], [-608.8041700422764, 1217.607874

In [49]:
# Get Pixel Size

# Get camera pixel size
camera_px = float(tree.find('.//ns:InstrumentDescription/ns:Cameras/ns:Camera/ns:PixelSizeX', ns).text)*10**6
# Get binning
binning = int(tree.find('.//ns:Experiment/ns:Exposures/ns:Exposure/ns:SimpleChannel/ns:CameraSetting/ns:BinningX', ns).text)
# Get objective magnification
M_objective = int(tree.find('.//ns:Experiment/ns:Exposures/ns:Exposure/ns:ObjectiveMagnification', ns).text)
M_factor = 1.087

pixel_size = (camera_px*binning)/(M_objective*M_factor) #um

In [ ]:
# Generate TileConfiguration for Stitching
import os
top_text = ['# Define the number of dimensions we are working on', 'dim = 2', '# Define the image coordinates (in pixels)']

config_dir = r"./TileConfigurations"
if not os.path.exists(config_dir):
    os.makedirs(config_dir)

for well in well_layout.keys():
    cur_file = os.path.join(config_dir, f'TileConfiguration_{well}.txt')

    fields = well_layout[well]

    with open(cur_file, 'w') as f:
        f.write('\n'.join(top_text))
        f.write('\n')
        for i, field in enumerate(fields):
            image_name = well+'f'+str(i+1).zfill(2)# if i<10 else well+'f'+str(i+1)
            x = field[0]/pixel_size
            y = -field[1]/pixel_size # Needs to be inverted for stitching in ImageJ

            f.write(image_name+'.tif'+f'; ; ({x}, {y})\n')
    f.close()


In [47]:
pixel_size

0.5933762649494021

In [ ]:
# Get field overlap

overlap = int(tree.find('.//ns:Experiment/ns:Sublayouts/ns:Sublayout/ns:Definition/ns:OverlapX', ns).text)
print(overlap)

In [ ]:
# Get GUID and measurement name

guid = tree.find('ns:MeasurementID', ns).text
measurement_name = tree.find('ns:InitialPlateName', ns).text
print(guid, measurement_name)

In [ ]:
# Get number of planes

num_planes = int(tree.find('.//ns:Stack/ns:Planes', ns).text)
print(num_planes)

# Just moving through the children of the tree to get an idea of what's going on

In [ ]:
for child in root:
    print(child.tag)

In [ ]:
experiment = root[7]
for child in experiment:
    print(child.tag)

In [ ]:
sublayouts = experiment[6]
print(len(sublayouts))
for child in sublayouts:
    print(child.tag)